In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from numpy import nan as NA
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

## Uploading Data Set

In [ ]:
drive.mount('/content/drive')

In [ ]:
import chardet
import glob



def isItUnicode(filename):
    with open(filename, 'rb') as f:
        encodingInfo = chardet.detect(f.read())
        if "UTF" not in encodingInfo['encoding']:
            print("This isn't Unicode! It's", encodingInfo['encoding'])
        else:
            print("Yep, it's Unicode.")

In [ ]:
isItUnicode('/content/drive/MyDrive/Notebooks/SeoulBikeData.csv')

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Notebooks/SeoulBikeData.csv',encoding='ISO-8859-1')

## Analysis


In [ ]:
df1.shape

In [ ]:
df1.describe().T

In [ ]:
df1.tail()

In [ ]:
def perc_miss(df):
  miss = df.isnull().sum().sort_values(ascending=False)
  miss = miss[miss.values != 0]
  tt_cels = np.product(df.shape)
  tt_missing = miss.sum()

  perc = round(((tt_missing/tt_cels) * 100),3)
  pr = 'Percent of miss:' + str(perc) + "%"
  _all = "In sum:" + str(tt_missing.sum())

  return pr,_all,miss



In [ ]:
perc_miss(df1)

In [ ]:
import missingno as mn
mn.matrix(df1,color=(0,0,0))

As you can see , the data set is clean

In [ ]:
df1.nunique().sort_values(ascending=False)

In [ ]:
for i in df1.columns:
  print(i,df1[i].unique())
  print('--------------')

In [ ]:
def scatt_y(df, y):
    df_col = df.drop(y, axis=1).columns
    n_cols = len(df_col)


    n_rows = (n_cols + 1) // 2
    fig, axs = plt.subplots(n_rows, 2, figsize=(14, n_rows * 5))

    if n_cols % 2 != 0:
        fig.delaxes(axs[-1, -1])


    for i, col in enumerate(df_col):
        ax = axs[i // 2, i % 2]
        ax.scatter(df[col], df[y], marker='o', c="pink")
        ax.set_xlabel(col)
        ax.set_ylabel(y)

    plt.tight_layout()
    plt.show()


In [ ]:
scatt_y(df1,'Rented Bike Count')

In [ ]:
from scipy.stats import norm

def all_hist(df):
    df_col = df.columns
    df_col = [i for i in df_col if df[i].dtype in ['int64', 'float64']]

    for i, col in enumerate(df_col):
        plt.figure(i)
        data = df[col]
        data.hist(bins=28, density=True, alpha=0.6, color='g')


        mu, std = norm.fit(data)


        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        p = norm.pdf(x, mu, std)

        plt.plot(x, p, 'k', linewidth=2)

        title = f"{col}, mu = {mu:.2f},  std = {std:.2f}"
        plt.title(title)

        plt.axvline(mu, color='k', linestyle='dashed', linewidth=1)


        plt.show()


In [ ]:

all_hist(df1)

In these graphs we can see a non-normal distribution in some features, but this is not a problem for the decision tree algorithm that we will use in the future

In [ ]:
col = list(df1.columns)

n_cols = 2
n_rows = (len(col) + n_cols - 1) // n_cols


fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(15, 20))


for ax in axes.flatten()[len(col):]:
    ax.remove()


axes_flat = axes.flatten()

for i, column_name in enumerate(col):

    sns.boxplot(df1[column_name], ax=axes_flat[i])
    axes_flat[i].set_title(column_name)  # Встановлюємо заголовок
    axes_flat[i].set_xticklabels(axes_flat[i].get_xticklabels(), rotation=45)  # Повертаємо мітки осей


plt.tight_layout()



we will reduce some range in some columns and remove some outliers further

In [ ]:
g = sns.FacetGrid(df1, col="Seasons", col_wrap=2, height=7, aspect=1.5)

g.map(sns.barplot, "Hour", "Rented Bike Count", color='r', order=sorted(df1['Hour'].unique()))

g.set_axis_labels("Година", "Кількість орендованих велосипедів")
g.set_titles("{col_name} сезон")

Obviously, there are fewer bike rentals in the winter season

In [ ]:
df1.columns

In [ ]:
def temp(x):
    if x < 0:
        return 0
    elif 0 <= x and x < 15:
        return 1
    elif 15 <= x and x < 25:
        return 3
    elif 25 <= x:
        return 4

In [ ]:
def rain_pw(x):
    if x <= 0.1:
        return 0
    elif 0.1 < x and x < 2.5:
        return 1
    elif 2.5 <= x and x < 7.6:
        return 2
    elif x >= 7.6:
        return 3

In [ ]:
def wind_pw(x):
    if x <= 0.3:
        return 0  # без вітра
    elif 0.3 < x and x <= 3.4:
        return 1  # слабкий
    elif 3.4 < x and x <= 8.0:
        return 2  # нормальний
    elif x > 8.0:
        return 3  # сильний

In [ ]:
df1['isTemp'] = df1['Temperature(°C)'].apply(temp)

In [ ]:
df1['isRain'] = df1['Rainfall(mm)'].apply(rain_pw)

In [ ]:
df1['isHoliday']=df1['Holiday'].map({'Holiday':1,'No Holiday':0})

In [ ]:
df1['isSnow'] = df1['Snowfall (cm)'].apply(lambda x: 1 if x > 0.5  else 0)

In [ ]:
def barp_hue(x,y,h,data):
	plt.figure(figsize=(11,7))
	sns.barplot(x=x,y=y,hue=h,data=data)

In [ ]:
h = ['isTemp','isRain','isHoliday','isSnow']
for i in h:
	barp_hue('Hour','Rented Bike Count',i,df1)

After viewing the graphs, it can be assumed that the high air temperature, the absence of rain and the absence of snow have a positive effect on the number of bicycle rentals in the city.



It can also be assumed that on non-holidays (which means rather on working days) at six o'clock there is a peak in the number of people renting bicycles (Given the publicly available data that the working day in South Korea ends at 5-6 o'clock in the evening, it can be assumed that people take them in order to get home after work or to other necessary places).

In [ ]:
df1.drop(h,inplace=True,axis = 1)

## FeaturesEngineering


As mentioned earlier we are going to reduce the spread range of some data and remove some outliers, which ones will be shown below

In [ ]:
fig,ax = plt.subplots(figsize=(12,12))

ax.scatter(x=df1['Rainfall(mm)'], y=df1['Rented Bike Count'], marker='o')
ax.set_xlabel('')
ax.set_ylabel('')

In [ ]:
def remove_outliers(df, columns):
    def find_outliers(series):
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return (series < lower_bound) | (series > upper_bound)

    for col in columns:
        if col in df.columns:
            outliers = find_outliers(df[col])
            df = df[~outliers]

    return df

In [ ]:
df1 = df1[df1['Rainfall(mm)'] <= 20]

In [ ]:
df1 = df1[df1['Snowfall (cm)'] <= 5.7]

In [ ]:
df1 = remove_outliers(df1,['Solar Radiation (MJ/m2)'])

In [ ]:
df1.shape

In [ ]:
df1.head()

In [ ]:
df1['Date'] = pd.to_datetime(df1['Date'], format='%d/%m/%Y')


In [ ]:
df1.drop_duplicates()

We plan to develop a new attribute for our dataset that will indicate the time of day as "light" or "dark". This attribute will be based on sunrise and sunset statistics for Seoul. After introducing this attribute, we analyze the effect on the accuracy of our model's predictions to understand if this time factor is significant for the behavior or phenomena we are trying to predict

In [ ]:
def is_dark(date, hour):
    sunset, sunrise = {
        1: (17, 7),
        2: (18, 7),
        3: (18, 6),
        4: (19, 6),
        5: (19, 5),
        6: (20, 5),
        7: (20, 5),
        8: (19, 6),
        9: (19, 6),
        10: (18, 7),
        11: (17, 7),
        12: (17, 7)
    }.get(date.month, (None, None))

    if sunset is not None:
        return hour >= sunset or hour < sunrise
    return False




In [ ]:
df1['Times of Day'] = ['Dark' if is_dark(date, hour) else 'Light' for date, hour in zip(df1['Date'], df1['Hour'])]

In [ ]:
df1

In [ ]:
# Підрахунок кількості кожної категорії
times_of_day_counts = df1['Times of Day'].value_counts()

# Створення кругової діаграми
plt.pie(x=times_of_day_counts, labels=times_of_day_counts.index, autopct='%1.1f%%', shadow=True, radius=1.5)


In [ ]:
df1.groupby('Times of Day').count()

Let's split the data column into three columns

In [ ]:
df1['Day'] = df1['Date'].dt.day
df1['Month'] = df1['Date'].dt.month
df1['Year'] = df1['Date'].dt.year

df1 = df1.drop('Date',axis=1)

In [ ]:
df1.head()

In [ ]:
for i in df1.columns:
    if df1[i].dtype == 'object':
        print(i, df1[i].unique())
        print('--------------')


**Here our task is to code some features**

In [ ]:
df_cp = df1.copy()

hot_col = ['Seasons','Holiday','Functioning Day','Times of Day']


hot = pd.get_dummies(df1[hot_col])
hot = hot.astype('float64')
df_cp = df_cp.drop(hot_col,axis=1)
df_cp = df_cp.join(hot)


In [ ]:
df_cp.columns

In [ ]:
df1 = df_cp.copy()

## Features Select

In [ ]:
y = df_cp['Rented Bike Count']

df_cp.drop('Rented Bike Count',inplace=True,axis=1)

X = df_cp

In [ ]:
from xgboost import XGBRegressor


model = XGBRegressor()

model.fit(X, y)

importance = model.feature_importances_

features_importances = [(i, v) for i, v in enumerate(importance)]
features_importances.sort(key=lambda x: x[1], reverse=True)


for i, v in features_importances:
    print('Feature: %d, Score: %.5f' % (i,v))

plt.figure(figsize=(12,12))

sorted_features = [X.columns[i] for i, _ in features_importances]
sorted_importance = [v for _, v in features_importances]
plt.bar(sorted_features, sorted_importance)
plt.xticks(rotation=90)

plt.show()

In [ ]:
from sklearn.feature_selection import RFE
# here we want only one final feature, we do this to produce a ranking
n_features_to_select = 10
rfe = RFE(model, n_features_to_select=n_features_to_select)
rfe.fit(X, y)


from operator import itemgetter
features = X.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)

In [ ]:
cor = df1.corr(numeric_only=True)


plt.figure(figsize=(18, 13))

sns.heatmap(cor, cmap='coolwarm', annot=True)



In [ ]:
df1['Functioning Day_Yes'].dtype

In [ ]:
df1.columns

In [ ]:
df1 = df1[df1['Functioning Day_No'] != 1]

In [ ]:
df1 = df1.copy()

We can safely remove this feature from our data set because it is obvious to predict

In [ ]:
df1.drop(['Functioning Day_No', 'Functioning Day_Yes'],axis=1,inplace=True)

In [ ]:
df1.shape

In [ ]:
df1['Holiday_Holiday'].dtype

After all the analysis using 3 feature selection methods, we can say that these features can be removed

In [ ]:
cols = ['Dew point temperature(°C)','Wind speed (m/s)','Visibility (10m)','Year','Times of Day_Dark','Times of Day_Light','Seasons_Autumn','Seasons_Spring','Seasons_Summer','Seasons_Winter']

df1.drop(cols,inplace=True,axis=1)

In [ ]:
df1.head()

## Model Building

In [ ]:
y = df1['Rented Bike Count']
df1.drop('Rented Bike Count',inplace=True,axis = 1)
X = df1

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
regressor= XGBRegressor(eval_metric='rmse')

In [ ]:
regressor

In [ ]:
'''
from sklearn.model_selection import GridSearchCV

param_grid = {"max_depth": [10],
              "n_estimators": [2000],
              "learning_rate": [0.05],
              "reg_alpha": [0.03,0.05,0.07],
              "reg_lambda": [13,15,17],

              }


search = GridSearchCV(regressor, param_grid, cv=5,n_jobs=-1).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)
'''

In [ ]:
regressor= XGBRegressor(
                          #learning_rate = search.best_params_["learning_rate"],
                          #n_estimators  = search.best_params_["n_estimators"],
                           #max_depth  = search.best_params_["max_depth"],
                           #reg_alpha =  search.best_params_["reg_alpha"],
                           #reg_lambda =  search.best_params_["reg_lambda"],

                          #MANUAL:

                        learning_rate = 0.03,
                          n_estimators  = 2000,
                          max_depth     = 10,
                          reg_alpha =  0.05,
                          reg_lambda = 15 ,
                          subsample = 0.7,
                          random_state = 42,
                           eval_metric='rmse')

regressor.fit(X_train, y_train)

Additional evaluation of the model by cross-validation for greater reliability

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [ ]:
params = {
    'learning_rate': 0.03,
    'max_depth': 10,
    'reg_alpha': 0.05,
    'reg_lambda': 15,
    'subsample': 0.7,
    'random_state': 42,
    'eval_metric': 'rmse'
}


In [ ]:
res = xgb.cv(
    params,
    dtrain,
    num_boost_round=1000,
    nfold=4,
    metrics={"rmse"},
    seed=0,
)


In [ ]:

print(res)
print("running cross validation, with preprocessing function")


In [ ]:
y_pred1 = regressor.predict(X_test)

In [ ]:
def reg_eval(y_test,y_pred):
  r2 = r2_score(y_test, y_pred)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)

  print("R2 : %f" % round((r2),3))
  print("MAE : %f" % round((mae),2))
  print("RMSE : %f" % round((rmse),2))
  print("MSE : %f" % round((mse),2))


In [ ]:
reg_eval(y_test,y_pred1)

Let's save our model in a pkl file

In [ ]:
import joblib
file_name = 'regressor_bikes.pkl'
joblib.dump(regressor, file_name)
files.download(file_name)